## Problem 4 - Implement SVM with SMO Solver:
SMO algo solves the SVM optimization problem by breaking it into smallest possible sub problems.

Dual SVM Problem:
$$
max \sum_{i=1}^n \alpha_i - \frac{1}{2} \sum_{i,j=1}^n \alpha_i \alpha_j y_i y_j K(x_i, x_j)
$$

under following conditions:
- $ 0 <= \alpha_i <= C $ for all i
- $ \sum_{i=1}^n \alpha_i y_i = 0 $

Idea: instead of solving for all alpha values at once - SMO
1. Picks two alpha values at a time ($ \alpha_i , \alpha_j $)
2. optimizes this pair and keeps others fixed
3. repeats the process with other $ \alpha $

In [7]:
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split